In [6]:
import sys
sys.path.append("../../src")

In [7]:
import numpy as np
import cv2
from cv2 import aruco
import pathlib

def calibrate_charuco(images: list, marker_length, square_length):
    '''Apply camera calibration using aruco.
    The dimensions are in cm.
    '''
    aruco_dict = aruco.Dictionary_get(aruco.DICT_4X4_50) 
    board = aruco.CharucoBoard_create(5, 7, square_length, marker_length, aruco_dict)
    arucoParams = aruco.DetectorParameters_create()

    counter, corners_list, id_list = [], [], []
    first = 0
    # Find the ArUco markers inside each image
    for image in images:
        img_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        corners, ids, rejected = aruco.detectMarkers(
            img_gray, 
            aruco_dict, 
            parameters=arucoParams
        )

        resp, charuco_corners, charuco_ids = aruco.interpolateCornersCharuco(
            markerCorners=corners,
            markerIds=ids,
            image=img_gray,
            board=board
        )
        # If a Charuco board was found, let's collect image/corner points
        # Requiring at least 20 squares
        if resp > 20:
            # Add these corners and ids to our calibration arrays
            corners_list.append(charuco_corners)
            id_list.append(charuco_ids)

    # Actual calibration
    ret, mtx, dist, rvecs, tvecs = aruco.calibrateCameraCharuco(
        charucoCorners=corners_list, 
        charucoIds=id_list, 
        board=board, 
        imageSize=img_gray.shape, 
        cameraMatrix=None, 
        distCoeffs=None)
    
    return [ret, mtx, dist, rvecs, tvecs]


def save_coefficients(mtx, dist, tmat, path): 
    '''Save the camera matrix and the distortion coefficients to given path/file.''' 
    cv_file = cv2.FileStorage(path, cv2.FILE_STORAGE_WRITE)
    cv_file.write('K', mtx)
    cv_file.write('D', dist)
    cv_file.write('T', tmat)
    # note you *release* you don't close() a FileStorage object
    cv_file.release()

def load_coefficients(path):
    '''Loads camera matrix and distortion coefficients.'''
    # FILE_STORAGE_READ
    cv_file = cv2.FileStorage(path, cv2.FILE_STORAGE_READ)

    # note we also have to specify the type to retrieve other wise we only get a
    # FileNode object back instead of a matrix
    camera_matrix = cv_file.getNode('K').mat()
    dist_matrix = cv_file.getNode('D').mat()
    transf_matrix = cv_file.getNode('T').mat()

    cv_file.release()
    return [camera_matrix, dist_matrix, transf_matrix]

In [8]:
def get_homogeneous(rvecs, tvecs):
    hom = np.identity(4)
    rmat, _ = cv2.Rodrigues(np.reshape(rvecs, -1))
    hom[0:3, 0:3] = rmat
    hom[0:3, 3] = np.squeeze(tvecs[0])
    return hom

### Calculate & save intr. & extr. matrix

In [11]:
from matplotlib import pyplot as plt
from models.dataset.DatasetHelper import openImageDataset


# Parameters
DATASET_PATH = "../../resources/images/uncalibrated/calibration_dataset.npz"

# Dimensions in cm
MARKER_LENGTH = 5.625
SQUARE_LENGTH = 7.5

# load images
realsenseColor, _, zividColor, _ = openImageDataset(DATASET_PATH)


# Calibrate 
ret_rs, mtx_rs, dist_rs, rvecs_rs, tvecs_rs = calibrate_charuco(
    realsenseColor,
    MARKER_LENGTH,
    SQUARE_LENGTH
)

ret_ziv, mtx_ziv, dist_ziv, rvecs_ziv, tvecs_ziv = calibrate_charuco(
    zividColor,
    MARKER_LENGTH,
    SQUARE_LENGTH
)

# Calculate both homogenous transformation matrixes
T_rs = get_homogeneous(rvecs_rs, tvecs_rs)
T_ziv = get_homogeneous(rvecs_ziv, tvecs_ziv)


# Save coefficients into a file
save_coefficients(mtx_rs, dist_rs, T_rs, '../../resources/calibrations/calibration_rs.yml')
save_coefficients(mtx_ziv, dist_ziv, T_ziv, '../../resources/calibrations/calibration_ziv.yml')

error: OpenCV(4.5.3) /home/conda/feedstock_root/build_artifacts/libopencv_1633800894050/work/opencv_contrib/modules/aruco/src/charuco.cpp:447: error: (-215:Assertion failed) _markerCorners.total() == _markerIds.getMat().total() && _markerIds.getMat().total() > 0 in function '_interpolateCornersCharucoLocalHom'


0.075 m
4x4 aruco a 1/8 bzw 6/8 * 0.075 

### Transform using o3d [http://www.open3d.org/docs/release/tutorial/geometry/transformation.html](test)

In [12]:
def transform_ziv_to_rs_image(ziv_img):
   _, _, T_rs = load_coefficients('../../resources/calibrations/calibration_rs.yml')
   _, _, T_ziv = load_coefficients('../../resources/calibrations/calibration_ziv.yml')
   return ziv_img.transform(np.linalg.inv(T_rs) @ T_ziv) 

In [13]:

dst = cv2.undistort(
    original, mtx_rs, dist, None, mtx_rs
)
cv2.imwrite('../../resources/images/calibrated/undist_rs.png', dst)

NameError: name 'original' is not defined